# Mood & Time Aware Content Recommendation System

---

## 1. Problem Definition & Objective

### a. Selected Project Track
**AI/ML - Personalized Content Recommendation System**

### b. Clear Problem Statement
Users often experience decision fatigue or find themselves consuming content that doesn't align with their current emotional state or time of day. Most platforms recommend content based solely on past history, ignoring the user's *immediate* mood. Anshika aims to bridge this gap by suggesting personalized activities and YouTube videos based on a real-time assessment of user mood (Focus, Relax, Bored, or Sad) and the current time of day.

### c. Real-world Relevance and Motivation
In an era of information overload, mental well-being is closely tied to digital consumption. Suggesting a high-intensity action movie to someone feeling 'Sad' might be counterproductive, whereas an uplifting music video could improve their mood. Anshika motivates balanced living by adapting the digital environment to the user's psychological state.

## 2. Data Understanding & Preparation

### a. Dataset Source
We use **Synthetic Data** generated through LLM simulation to represent user usage patterns (Time of Day, Past Moods, Content Types). For content recommendations, we use the **YouTube Data API** (simulated via AI flows) to fetch relevant video metadata.

### b. Data Loading and Exploration
Below is a simulation of the data structure used for mood prediction.

In [7]:
import pandas as pd
import json

# Mock dataset representing user history
data = {
    "time_of_day": ["Morning", "Afternoon", "Evening", "Night", "Morning"],
    "past_moods": ["Focus", "Bored", "Relax", "Sad", "Focus"],
    "content_types": ["Coding Tutorial", "Comedy", "Lo-fi Music", "Uplifting Pop", "Productivity Podcast"]
}
df = pd.DataFrame(data)
print("Sample User Activity Data:")
print(df.head())

Sample User Activity Data:
  time_of_day past_moods         content_types
0     Morning      Focus       Coding Tutorial
1   Afternoon      Bored                Comedy
2     Evening      Relax           Lo-fi Music
3       Night        Sad         Uplifting Pop
4     Morning      Focus  Productivity Podcast


### c. Cleaning, Preprocessing, Feature Engineering
The input data is structured into a JSON format expected by the Genkit `suggestMoodFlow`. Feature engineering involves categorical encoding of 'Time of Day' and 'Moods' which the LLM processes as semantic tokens.

### d. Handling Missing Values
Missing history is handled by defaulting to a 'Neutral/Bored' state or providing a diverse mix of content until a pattern is established.

## 3. Model / System Design

### a. AI Technique Used
**Hybrid Approach**: Large Language Models (LLM) using Google Gemini for semantic reasoning (Mood classification and Content matching) integrated with a structured pipeline via **Firebase Genkit** for orchestration.

### b. Architecture or Pipeline Explanation
1. **Input Layer**: Captures current time and recent user interactions.
2. **Mood Flow (`auto-mood-suggestion`)**: LLM analyzes history to predict current mood state.
3. **Recommendation Flow (`youtube-video-suggester`)**: LLM uses predicted mood to search/generate 9 relevant video recommendations.
4. **Output Layer**: Dynamic Dashboard updates UI with personalized suggestions.

### c. Justification of Design Choices
- **Genkit**: Provides strongly typed AI flows, making it robust for production use.
- **LLM for Prediction**: Unlike traditional ML classifiers, LLMs can interpret complex semantic relationships between varied content types (e.g., 'Lo-fi' suggesting a 'Relax' mood).

## 4. Core Implementation

### a. Model Training / Inference Logic
Since we use a pre-trained LLM (Gemini 1.5), "training" is replaced by fine-tuning prompts. Below is the simulation of the Inference Logic.

In [8]:
def simulate_mood_inference(time_of_day, past_moods):
    """Simulates the logic in src/ai/flows/auto-mood-suggestion.ts"""
    if time_of_day == "Night" and "Relax" in past_moods:
        return {"suggestedMood": "Relax", "confidence": 0.92}
    elif "Bored" in past_moods:
        return {"suggestedMood": "Focus", "confidence": 0.75} # Pivot to focus for productivity
    return {"suggestedMood": "Relax", "confidence": 0.50}

current_prediction = simulate_mood_inference("Night", ["Focus", "Relax"])
print(f"Inference Result: {current_prediction}")

Inference Result: {'suggestedMood': 'Relax', 'confidence': 0.92}


### b. Prompt Engineering
The core of Anshika lies in its prompt structure:
```
System: You are an empathetic content curator.
Input: {timeOfDay, pastMoods, contentHistory}
Output: Suggest one of [Focus, Relax, Bored, Sad].
```
For video suggestions, we use **Few-Shot Prompting** to ensure the JSON output contains valid YouTube metadata format.

### c. Recommendation Pipeline
The pipeline executes `suggestMoodFlow` -> `suggestVideosFlow` sequentially.

In [9]:
def get_recommendations(mood):
    # Simulated response from src/ai/flows/youtube-video-suggester.ts
    return [
        {"title": f"Uplifting {mood} Music", "id": "dQw4w9WgXcQ", "desc": "Best curated list."},
        {"title": f"How to stay {mood}", "id": "ABC123XYZ", "desc": "Expert tips."}
    ]

print(f"Recommendations for {current_prediction['suggestedMood']}:")
print(json.dumps(get_recommendations(current_prediction['suggestedMood']), indent=2))

Recommendations for Relax:
[
  {
    "title": "Uplifting Relax Music",
    "id": "dQw4w9WgXcQ",
    "desc": "Best curated list."
  },
  {
    "title": "How to stay Relax",
    "id": "ABC123XYZ",
    "desc": "Expert tips."
  }
]


## 5. Evaluation & Analysis

### a. Metrics Used
- **Quantitative**: Confidence score (0-1) returned by the model.
- **Qualitative**: Relevance check—does 'Sad' mood produce 'Uplifting' content?

### b. Sample Outputs
Input: `{ "mood": "Sad" }` -> AI returns "Uplifting Pop" and "Cute Animal Compilations".

### c. Performance Analysis and Limitations
- **Performance**: LLM inference takes ~1-2s, manageable for real-time app use.
- **Limitations**: Highly dependent on the quality of user input history; occasional hallucination of YouTube video IDs (mitigated by Genkit schema validation).

## 6. Ethical Considerations & Responsible AI

### a. Bias and Fairness Considerations
The system avoids suggesting harmful or sensitive content in response to negative moods (like 'Sad'). It prioritizes clinically safe, general-purpose uplifting content.

### b. Dataset Limitations
Synthetic data may not capture the full complexity of human emotions. The model is restricted to 4 major categories to maintain accuracy.

### c. Responsible use of AI tools
Anshika includes a disclaimer that it is a *companion* app and not a substitute for professional mental health advice.

## 7. Conclusion & Future Scope

### a. Summary of Results
Anshika successfully implements a mood-aware content pipeline using Genkit and Gemini. It demonstrates that combining time-of-day data with semantic user history allows for highly personalized digital experiences.

### b. Possible Improvements and Extensions
- **Real API Integration**: Connect to YouTube Data API for real-time video validation.
- **Multi-Modal Input**: Use facial expression analysis (via camera) to determine mood more accurately.
- **Extended Ecosystem**: Suggest podcasts, Spotify playlists, and Pomodoro timers integrated into the flow.